In [1]:
import pyomo.environ as pyo
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import re
import datetime
import random
import os

In [2]:
# load day ahead price data from E:\day-ahead price\Data\ISO_New_England\day_ahead_price
path = r'E:\day-ahead price\Data\ISO_New_England\day_ahead_price'
os.chdir(path)
files = os.listdir(path)
df = pd.DataFrame()
for file in files:
    delimiter = ','
    max_columns = max(open(file), key=lambda x: x.count(delimiter)).count(delimiter)
    df_tmp = pd.read_csv(file, header=None, names=list(range(max_columns+1)))
    # print(df_tmp)
    df = pd.concat([df, df_tmp])
df.columns = df.iloc[4]
df = df[df['H'] == 'D'].drop(['H'], axis = 1).reset_index(drop = True)
# average price by Date and Hour Ending
tmp = []
df['Hour Ending'] = df['Hour Ending'].astype(str)
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Locational Marginal Price'] = df['Locational Marginal Price'].astype(float)
df['Energy Component'] = df['Energy Component'].astype(float)
df['Congestion Component'] = df['Congestion Component'].astype(float)
df['Marginal Loss Component'] = df['Marginal Loss Component'].astype(float)
df_group = df.groupby(['Date', 'Hour Ending'])
for [Date, Hour_Ending], group in df_group:
    Locational_Marginal_Price = group['Locational Marginal Price'].mean() 
    Energy_Component = group['Energy Component'].mean()
    Congestion_Component = group['Congestion Component'].mean()
    Marginal_Loss_Component = group['Marginal Loss Component'].mean()
    tmp.append([Date, Hour_Ending, Locational_Marginal_Price, Energy_Component, Congestion_Component, Marginal_Loss_Component])
df_price = pd.DataFrame(tmp, columns = ['Date', 'Hour Ending', 'Locational Marginal Price', 'Energy Component', 'Congestion Component', 'Marginal Loss Component'])
# df_price lack a line of data in Date 2023-03-12, Hour Ending 3, so we need to add this line by average of Hour Ending 2 and Hour Ending 4
lost_data = pd.DataFrame({'Date': '2023-03-12', 'Hour Ending': '03', 'Locational Marginal Price': (df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '02')]['Locational Marginal Price'].values[0] + df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '04')]['Locational Marginal Price'].values[0])/2, 'Energy Component': (df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '02')]['Energy Component'].values[0] + df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '04')]['Energy Component'].values[0])/2, 'Congestion Component': (df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '02')]['Congestion Component'].values[0] + df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '04')]['Congestion Component'].values[0])/2, 'Marginal Loss Component': (df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '02')]['Marginal Loss Component'].values[0] + df_price[(df_price['Date'] == '2023-03-12') & (df_price['Hour Ending'] == '04')]['Marginal Loss Component'].values[0])/2}, index = [0])
df_price = pd.concat([df_price, lost_data]).sort_values(by = ['Date', 'Hour Ending']).reset_index(drop = True)
# delete the line of data in Date 2023-11-05, whose Hour Ending = '02X' 
df_price = df_price.drop(df_price[(df_price['Date'] == '2023-11-05') & (df_price['Hour Ending'] == '02X')].index).reset_index(drop = True)
# delete the line of data in Date 2022-11-06, whose Hour Ending = '02X' 
df_price = df_price.drop(df_price[(df_price['Date'] == '2022-11-06') & (df_price['Hour Ending'] == '02X')].index).reset_index(drop = True)
df_price['Hour Ending'] = df_price['Hour Ending'].astype(int)
df_price['Date'] = pd.to_datetime(df_price['Date'])
df_price

,Date,Hour Ending,Locational Marginal Price,Energy Component,Congestion Component,Marginal Loss Component
0,2022-01-01,1,32.124229,32.35,-0.089337,-0.136434
1,2022-01-01,2,32.057678,32.31,-0.111808,-0.140514
2,2022-01-01,3,30.726426,30.85,0.000000,-0.123574
3,2022-01-01,4,29.569668,29.69,0.000000,-0.120332
4,2022-01-01,5,30.738126,30.86,0.000000,-0.121874
...,...,...,...,...,...,...
17610,2024-01-04,20,56.589893,57.31,-0.000592,-0.719515
17611,2024-01-04,21,55.407730,56.06,0.000000,-0.652270
17612,2024-01-04,22,52.167023,52.78,-0.036324,-0.576653
17613,2024-01-04,23,51.075979,51.71,-0.096390,-0.537632


In [2]:
# load real load data from E:\day-ahead price\Data\ISO_New_England\real_load
path = r'E:\day-ahead price\Data\ISO_New_England\real_load'
os.chdir(path)
files = os.listdir(path)
df_real_load = pd.DataFrame()
for file in files:
    delimiter = ','
    max_columns = max(open(file), key=lambda x: x.count(delimiter)).count(delimiter)
    df_tmp = pd.read_csv(file, header=None, names=list(range(max_columns+1)))
    df_tmp.columns = df_tmp.iloc[4]
    df_tmp = df_tmp[df_tmp['H'] == 'D'].iloc[:, 1:].reset_index(drop=True)
    df_tmp['Date'] = pd.to_datetime(df_tmp['Date'])
    df_tmp = df_tmp[df_tmp['Hour Ending'] != '02X']
    df_tmp['Hour Ending'] = df_tmp['Hour Ending'].astype(int)
    df_tmp['Day-Ahead Cleared Demand'] = df_tmp['Day-Ahead Cleared Demand'].astype(float)
    df_real_load = pd.concat([df_real_load, df_tmp], axis=0)
# df_real_load lack a line of data in Date 2023-03-12, Hour Ending 3, so we need to add this line by average of Hour Ending 2 and Hour Ending 4
df_tmp = df_real_load[(df_real_load['Date'] == '2023-03-12') & (df_real_load['Hour Ending'] == 2)].reset_index(drop=True)
df_tmp['Hour Ending'] = 3
df_tmp['Day-Ahead Cleared Demand'] = (df_real_load[(df_real_load['Date'] == '2023-03-12') & (df_real_load['Hour Ending'] == 2)]['Day-Ahead Cleared Demand'].values + df_real_load[(df_real_load['Date'] == '2023-03-12') & (df_real_load['Hour Ending'] == 4)]['Day-Ahead Cleared Demand'].values) / 2
df_real_load = pd.concat([df_real_load, df_tmp], axis=0).reset_index(drop=True).sort_values(by=['Date', 'Hour Ending']).reset_index(drop=True)
df_real_load

4,Date,Hour Ending,Day-Ahead Cleared Demand
0,2022-10-01,1,9518.0
1,2022-10-01,2,9639.0
2,2022-10-01,3,9445.0
3,2022-10-01,4,9317.0
4,2022-10-01,5,9460.0
...,...,...,...
11059,2024-01-04,20,16328.0
11060,2024-01-04,21,15697.0
11061,2024-01-04,22,15034.0
11062,2024-01-04,23,13578.0


In [3]:
# df_real_load['Day-Ahead Cleared Demand'].mean()*24

301468.8579175705

In [4]:
def get_price_data(tgt_dt):
    df_1 = df_price[df_price['Date'] == tgt_dt].reset_index(drop=True)
    return df_1

def get_power_data(tgt_dt):
    df_1 = df_real_load[df_real_load['Date'] == tgt_dt].reset_index(drop=True)
    return df_1

In [5]:
def optimize_power_price(tgt_dt, MAE = 10):
    
    # 创建模型
    model = pyo.ConcreteModel()

    # 定义集合
    model.T = pyo.RangeSet(24)  # 一天96个15分钟的时间间隔

    # 参数
    soc_start = 5000  # 初始SOC
    soc_end = 5000  # 最终SOC
    charge_efficiency = 0.95  # 充电效率
    discharge_efficiency = 0.95  # 放电效率
    P_MAX = 5000  # 最大充放电功率
    # E_MAX = 4  # 储能容量
    # load = 75  # 固定负荷
    soc_min = 200  # SOC最小值
    soc_max = 10000  # SOC最大值
    load_MAX = 30000  # 最大负荷
    real_price = get_price_data(tgt_dt)['Locational Marginal Price'].values
    real_power = get_power_data(tgt_dt)['Day-Ahead Cleared Demand'].values
    noise = np.random.uniform(low = - MAE * 2, high = MAE * 2, size=24)
    prediced_price = real_price + noise
    prediced_price[prediced_price < 0] = 0
    # print(np.shape(prediced_price), np.shape(real_power), np.shape(noise))

    # 变量
    model.charge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, P_MAX))
    model.discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, P_MAX))
    model.soc = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(soc_min, soc_max))
    model.power = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, load_MAX))
    model.is_charging = pyo.Var(model.T, within=pyo.Binary)

    # 目标函数
    def cost_function(model):
        return sum(
            model.power[t] * prediced_price[t-1] for t in model.T
        )
    model.cost = pyo.Objective(rule=cost_function, sense=pyo.minimize)

    # 约束条件
    def soc_constraints(model, t):
        if t == 0:
            return True
        if t == 1:
            return model.soc[t] == soc_start + (model.charge[t] * charge_efficiency - model.discharge[t] * discharge_efficiency)
        else:
            return model.soc[t] == model.soc[t-1] + (model.charge[t] * charge_efficiency - model.discharge[t] * discharge_efficiency)
    model.soc_con = pyo.Constraint(model.T, rule=soc_constraints)

    def soc_final_constraint(model):
        return model.soc[24] == soc_end
    model.soc_final_con = pyo.Constraint(rule=soc_final_constraint)

    def power_constraint(model, t):
        return model.power[t] - model.charge[t] + model.discharge[t] == real_power[t-1]
    model.power_con = pyo.Constraint(model.T, rule=power_constraint)

    def charging_discharging_constraint(model, t):
        return model.charge[t] <= model.is_charging[t] * P_MAX 
    model.charging_con = pyo.Constraint(model.T, rule=charging_discharging_constraint)

    def discharging_charging_constraint(model, t): 
        return model.discharge[t] <= (1 - model.is_charging[t]) * P_MAX 
    model.discharging_con = pyo.Constraint(model.T, rule=discharging_charging_constraint)

    # 求解模型
    solver = pyo.SolverFactory('cplex')
    results = solver.solve(model)

    optimize_power = [pyo.value(model.power[t]) for t in model.T]

    return optimize_power

optimize_power_price(datetime.datetime(2023, 4, 1))

[6113.0,
 16127.0,
 5833.1578947368425,
 15260.0,
 5463.0,
 15945.0,
 16057.0,
 11491.0,
 12180.0,
 7601.0,
 7612.0,
 17359.0,
 11725.21052631579,
 6318.0,
 11393.78947368421,
 16197.0,
 16804.0,
 11982.0,
 12144.21052631579,
 8000.0,
 7903.0,
 17145.0,
 6692.631578947368,
 14997.0]

In [6]:
def calculate_alpha(tgt_dt, MAE = 10):
    zero_mae_power = optimize_power_price(tgt_dt, 0)
    zero_mae_power = np.array(zero_mae_power)
    real_price = get_price_data(tgt_dt)['Locational Marginal Price'].values
    real_power = get_power_data(tgt_dt)['Day-Ahead Cleared Demand'].values
    baseline = np.dot(real_price, real_power) - np.dot(real_price, zero_mae_power)
    monte_carlo_times = 100
    monte_carlo_list = []
    revenue_list = []
    for _ in range(monte_carlo_times):
        optimize_power = optimize_power_price(tgt_dt, MAE)
        optimize_power = np.array(optimize_power)
        # print(np.dot(real_price, (optimize_power - zero_mae_power)))
        monte_carlo_list.append(np.dot(real_price, (optimize_power - zero_mae_power)))
        revenue = np.dot(real_price, real_power) - np.dot(real_price, optimize_power)
        revenue_list.append(revenue)
    alpha = np.mean(monte_carlo_list) / baseline
    revenue_value = np.mean(revenue_list)
    revenue_percent = revenue_value / baseline
    return alpha, np.mean(monte_carlo_list), baseline, revenue, revenue_percent

calculate_alpha(datetime.datetime(2023, 4, 1))


(0.6083534299040876,
 247277.5723003071,
 406470.2525623841,
 99300.03113782872,
 0.3916465700959122)

In [7]:
def calculate_alpha_list(start_dt, end_dt, MAE = 10):
    date_list = pd.date_range(start_dt, end_dt)
    alpha_list = []
    reduced_cost_list = []
    baseline_cost_list = []
    revenue_list = []
    revenue_percent_list = []
    for date in date_list:
        try:
            alpha, reduced_cost, baseline_cost, revenue, revenue_percent = calculate_alpha(date, MAE)
            alpha_list.append(alpha)
            reduced_cost_list.append(reduced_cost)
            baseline_cost_list.append(baseline_cost)
            revenue_list.append(revenue)
            revenue_percent_list.append(revenue_percent)
        except:
            continue
    
    return alpha_list, reduced_cost_list, baseline_cost_list, revenue_list, revenue_percent_list

alpha_list, reduced_cost_list, baseline_cost_list, revenue_list, revenue_percent_list = calculate_alpha_list(datetime.datetime(2023, 1, 1), datetime.datetime(2023, 12, 31), 12.119805)
print(np.mean(alpha_list), np.mean(reduced_cost_list), np.mean(baseline_cost_list), np.mean(revenue_list), np.mean(revenue_percent_list))

0.6388718464061098 216772.30207710495 418887.4602248324 200604.49650137697 0.36112815359389017
